# Care Trading Companion - Supervised Machine learning project

This project uses supervised machine learning to predict used car prices based on historical sales data.


In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os
import re

## Download the Used Cars dataset from kaggle

In [2]:
def load_dataset():
    try:
        # Download dataset from Kaggle
        path = kagglehub.dataset_download("andreinovikov/used-cars-dataset")
        print("Dataset downloaded to:", path)

        # Find the CSV file inside the downloaded folder
        csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]

        if len(csv_files) == 0:
            raise FileNotFoundError("No CSV file found in the dataset folder.")

        csv_path = os.path.join(path, csv_files[0])

        # Load with pandas
        df = pd.read_csv(csv_path)
        print("Loaded CSV:", csv_path)
        print("Rows:", df.shape[0], "Columns:", df.shape[1])

        return df

    except Exception as e:
        print("Error loading dataset:", e)
        return None

## Load Dataset

In [3]:
df = load_dataset()
df.head()

Dataset downloaded to: C:\Users\dani\.cache\kagglehub\datasets\andreinovikov\used-cars-dataset\versions\1
Loaded CSV: C:\Users\dani\.cache\kagglehub\datasets\andreinovikov\used-cars-dataset\versions\1\cars.csv
Rows: 762091 Columns: 20


,manufacturer,model,year,mileage,engine,transmission,drivetrain,fuel_type,mpg,exterior_color,interior_color,accidents_or_damage,one_owner,personal_use_only,seller_name,seller_rating,driver_rating,driver_reviews_num,price_drop,price
0,Acura,ILX Hybrid 1.5L,2013,92945.0,"1.5L I-4 i-VTEC variable valve control, engine...",Automatic,Front-wheel Drive,Gasoline,39-38,Black,Parchment,0.0,0.0,0.0,Iconic Coach,NaN,4.4,12.0,300.0,13988.0
1,Acura,ILX Hybrid 1.5L,2013,47645.0,1.5L I4 8V MPFI SOHC Hybrid,Automatic CVT,Front-wheel Drive,Hybrid,39-38,Gray,Ebony,1.0,1.0,1.0,Kars Today,NaN,4.4,12.0,NaN,17995.0
2,Acura,ILX Hybrid 1.5L,2013,53422.0,1.5L I4 8V MPFI SOHC Hybrid,Automatic CVT,Front-wheel Drive,Hybrid,39-38,Bellanova White Pearl,Ebony,0.0,1.0,1.0,Weiss Toyota of South County,4.3,4.4,12.0,500.0,17000.0
3,Acura,ILX Hybrid 1.5L,2013,117598.0,1.5L I4 8V MPFI SOHC Hybrid,Automatic CVT,Front-wheel Drive,Hybrid,39-38,Polished Metal Metallic,NaN,0.0,1.0,1.0,Apple Tree Acura,NaN,4.4,12.0,675.0,14958.0
4,Acura,ILX Hybrid 1.5L,2013,114865.0,1.5L I4 8V MPFI SOHC Hybrid,Automatic CVT,Front-wheel Drive,Hybrid,39-38,NaN,Ebony,1.0,0.0,1.0,Herb Connolly Chevrolet,3.7,4.4,12.0,300.0,14498.0


## Explore Dataset

In [4]:
# Data structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 762091 entries, 0 to 762090
Data columns (total 20 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   manufacturer         762091 non-null  object 
 1   model                762091 non-null  object 
 2   year                 762091 non-null  int64  
 3   mileage              761585 non-null  float64
 4   engine               747041 non-null  object 
 5   transmission         752187 non-null  object 
 6   drivetrain           740529 non-null  object 
 7   fuel_type            739164 non-null  object 
 8   mpg                  620020 non-null  object 
 9   exterior_color       753232 non-null  object 
 10  interior_color       705116 non-null  object 
 11  accidents_or_damage  737879 non-null  float64
 12  one_owner            730608 non-null  float64
 13  personal_use_only    737239 non-null  float64
 14  seller_name          753498 non-null  object 
 15  seller_rating    

### Missing values in %

In [5]:
missing_percent = ((df.isnull().sum() / len(df)) * 100).sort_values(ascending=False)
missing_percent.round(2)

price_drop             46.19
seller_rating          28.08
mpg                    18.64
interior_color          7.48
driver_rating           4.15
one_owner               4.13
personal_use_only       3.26
accidents_or_damage     3.18
fuel_type               3.01
drivetrain              2.83
engine                  1.97
transmission            1.30
exterior_color          1.16
seller_name             1.13
mileage                 0.07
driver_reviews_num      0.00
manufacturer            0.00
model                   0.00
year                    0.00
price                   0.00
dtype: float64

### Number of unique values in text columns

In [6]:
# Select only text/categorical columns
object_cols = df.select_dtypes(include=['object']).columns

unique_counts = {}

for col in object_cols:
    unique_counts[col] = df[col].nunique()

unique_counts = pd.Series(unique_counts).sort_values(ascending=False)

unique_counts

seller_name       18254
model             12187
exterior_color     7681
engine             6903
interior_color     4679
transmission       1313
mpg                 865
fuel_type            36
drivetrain           33
manufacturer         30
dtype: int64

### Fuel type unique values

In [7]:
unique_fuel_types = df['fuel_type'].dropna().unique()
sorted_fuel_types = sorted(unique_fuel_types)

sorted_fuel_types

['Automatic',
 'B',
 'Bi-Fuel',
 'Bio Diesel',
 'Biodiesel',
 'Compressed Natural Gas',
 'Diesel',
 'Diesel Fuel',
 'E85 Fl',
 'E85 Flex Fuel',
 'Electric',
 'Electric Fuel System',
 'Flex Fuel',
 'Flex Fuel Capability',
 'Flexible Fuel',
 'G',
 'Gas',
 'Gas/Electric Hybrid',
 'Gaseous',
 'Gasoline',
 'Gasoline Fuel',
 'Gasoline/Mild Electric Hybrid',
 'Hybrid',
 'Hybrid Fuel',
 'Hydrogen Fuel Cell',
 'Natural Gas',
 'Other',
 'PHEV',
 'Plug-In Electric/Gas',
 'Plug-In Hybrid',
 'Premium',
 'Premium (Required)',
 'Premium Unleaded',
 'Regular Unleaded',
 'Unknown',
 'Unspecified']

### Price statistics

In [8]:
price_stats = df['price'].describe()

count = int(price_stats['count'])
mean = price_stats['mean']
std = price_stats['std']
min_price = price_stats['min']
q1 = price_stats['25%']
median = price_stats['50%']
q3 = price_stats['75%']
max_price = price_stats['max']

low_count = (df['price'] < 1000).sum()
high_count = (df['price'] > 200000).sum()

low_percent = (low_count / len(df)) * 100
high_percent = (high_count / len(df)) * 100

print("=== Price Statistics Summary ===")
print(f"Total entries: {count:,}")
print(f"Average price: ${mean:,.0f}")
print(f"Median price:  ${median:,.0f}")
print(f"Normal price range (middle 50%): ${q1:,.0f} → ${q3:,.0f}")
print(f"Minimum price found: ${min_price:,.0f}")
print(f"Maximum price found: ${max_price:,.0f}")
print(f"Standard deviation: ${std:,.0f}  (VERY HIGH — indicates extreme outliers)")

print(f"Cars priced under $1,000:      {low_count:,}  ({low_percent:.2f}%)")
print(f"Cars priced over $200,000:     {high_count:,}  ({high_percent:.2f}%)")
print("==================================")

=== Price Statistics Summary ===
Total entries: 762,091
Average price: $36,489
Median price:  $27,989
Normal price range (middle 50%): $19,583 → $39,488
Minimum price found: $1
Maximum price found: $1,000,000,000
Standard deviation: $1,984,183  (VERY HIGH — indicates extreme outliers)
Cars priced under $1,000:      39  (0.01%)
Cars priced over $200,000:     1,399  (0.18%)


### Milage statistics

In [9]:
m = df['mileage'].describe()

count = int(m['count'])
mean = m['mean']
std = m['std']
min_m = m['min']
q1 = m['25%']
median = m['50%']
q3 = m['75%']
max_m = m['max']

low_count = (df['mileage'] == 0).sum()
high_count = (df['mileage'] > 200000).sum()

low_percent = (low_count / len(df)) * 100
high_percent = (high_count / len(df)) * 100

print("=== Mileage Statistics Summary ===")
print(f"Total entries: {count:,}")
print(f"Average mileage: {mean:,.0f} miles")
print(f"Median mileage:  {median:,.0f} miles")
print(f"Normal range (middle 50%): {q1:,.0f} → {q3:,.0f} miles")
print(f"Minimum mileage found: {min_m:,.0f} miles")
print(f"Maximum mileage found: {max_m:,.0f} miles")
print(f"Standard deviation: {std:,.0f} miles")
print(f"Cars mileage with 0:      {low_count:,}  ({low_percent:.2f}%)")
print(f"Cars mileage over 200,000:     {high_count:,}  ({high_percent:.2f}%)")
print("==================================")

=== Mileage Statistics Summary ===
Total entries: 761,585
Average mileage: 55,782 miles
Median mileage:  45,596 miles
Normal range (middle 50%): 23,287 → 78,365 miles
Minimum mileage found: 0 miles
Maximum mileage found: 1,119,067 miles
Standard deviation: 43,558 miles
Cars mileage with 0:      1,564  (0.21%)
Cars mileage over 200,000:     6,244  (0.82%)


### Year statistics

In [10]:
y = df['year'].describe()

count = int(y['count'])
mean = y['mean']
std = y['std']
min_y = y['min']
q1 = y['25%']
median = y['50%']
q3 = y['75%']
max_y = y['max']

# Outlier checks — adjust as needed
old_count = (df['year'] < 1980).sum()
future_count = (df['year'] > 2025).sum()     

old_percent = (old_count / len(df)) * 100
future_percent = (future_count / len(df)) * 100

print("=== Year Statistics Summary ===")
print(f"Total entries: {count:,}")
print(f"Average year: {mean:,.0f}")
print(f"Median year:  {median:,.0f}")
print(f"Normal range (middle 50%): {q1:,.0f} → {q3:,.0f}")
print(f"Minimum year found: {min_y:,.0f}")
print(f"Maximum year found: {max_y:,.0f}")
print(f"Standard deviation: {std:,.0f}")
print(f"Cars older than 1980:      {old_count:,}  ({old_percent:.3f}%)")
print(f"Cars from future (>2025):  {future_count:,}  ({future_percent:.3f}%)")
print("================================")


=== Year Statistics Summary ===
Total entries: 762,091
Average year: 2,018
Median year:  2,019
Normal range (middle 50%): 2,016 → 2,021
Minimum year found: 1,915
Maximum year found: 2,024
Standard deviation: 5
Cars older than 1980:      2,253  (0.296%)
Cars from future (>2025):  0  (0.000%)


### Findings
- With 20 columns and over 762.000 records, this dataset is sufficient for our used-car price estimation.

- Core features are clean and have no missing values: ***manufacturer***, ***model***, ***year***, ***mileage***, ***price***

- The target ***price*** is complete, but contains extreme outliers ($0, $1B) that must be removed.

- Some text columns have an extremely large number of unique values, making them difficult or impossible to one-hot encode 

- ***Mileage*** includes unrealistic values (0 miles and >1,100,000 miles), indicating outliers that require cleaning.

- ***Milage***: Some cars have 0 miles,  

- ***price_drop*** has more than 46% missing values -> should be dropped

- ***seller_rating*** is missing 28% -> probably drop

- ***mpg*** is missing 18.64%, and is text format with 865 unique values -> likely drop

- ***engine*** column will need parsing: "1.5L I-4 i-VTEC variable valve control", -> Could extract engine size, and horse power

- Noise columns should be dropped: ***seller_name***, ***seller_rating***, ***price_drop***



In [11]:
model_counts = df["model"].value_counts()
model_counts.head(50)

model
Fusion SE                    3172
Sportage LX                  2873
Corolla LE                   2836
GLC 300 Base 4MATIC          2718
Sentra SV                    2652
Optima LX                    2650
Explorer XLT                 2542
Rogue SV                     2526
Tundra SR5                   2471
Sorento LX                   2461
Odyssey EX-L                 2391
RX 350 Base                  2385
Forte LXS                    2377
Wrangler Sport               2373
Focus SE                     2353
Edge SEL                     2285
F-150 XLT                    2279
Escape SE                    2232
Renegade Latitude            2232
Encore Preferred             2139
Grand Cherokee Limited       2081
Pacifica Touring-L           2057
Tiguan 2.0T SE               2047
Highlander XLE               2030
Frontier SV                  2008
Ranger XLT                   1921
CX-5 Touring                 1918
Grand Caravan SXT            1907
Malibu LT                    1905
C-Class 

## Clean dataset

In [12]:
print("Original shape:", df.shape)

# ===================== 1. PRICE: 2nd–98th percentile =====================
p_low, p_high = df["price"].quantile([0.02, 0.98])

df = df[(df["price"] >= p_low) & (df["price"] <= p_high)]
print("After percentile price filter:", df.shape)

# ===================== 2. MILEAGE: 2nd–98th percentile =====================
m_low, m_high = df["mileage"].quantile([0.02, 0.98])

df = df[(df["mileage"] >= m_low) & (df["mileage"] <= m_high)]
print("After percentile mileage filter:", df.shape)


# 3. Clean YEAR outliers
# Remove cars before 1980
df = df[(df['year'] >= 1980)]
print("After year filter:", df.shape)

# 4. Manufacturer to lowercase
df['manufacturer'] = df['manufacturer'].str.lower().str.strip()


# 5. Drop columns that are not useful or have too many missing values
cols_to_drop = [col for col in ['seller_name', 'seller_rating', 'price_drop', 'driver_rating', 'driver_reviews_num', "interior_color"] if col in df.columns]
df = df.drop(columns=cols_to_drop)
print("After dropping useless/high-missing columns:", df.shape)
print("Dropped columns:", cols_to_drop)

# Drop model and colors for now,
#df = df.drop(columns=['model'])

# 6. Convert MPG strings to a single numeric value (midpoint if range)
def convert_mpg_value(mpg):
    if pd.isna(mpg):
        return np.nan

    mpg = str(mpg).strip()

    # Grab all numbers (works for "30-35", "30 - 35", "32")
    nums = re.findall(r'\d+\.?\d*', mpg)

    if len(nums) == 2:
        # low-high -> average
        low, high = map(float, nums)
        return (low + high) / 2
    elif len(nums) == 1:
        # single number -> just use it
        return float(nums[0])
    else:
        # weird format -> treat as missing
        return np.nan

if 'mpg' in df.columns:
    df['mpg_avg'] = df['mpg'].apply(convert_mpg_value)
    df = df.drop(columns=['mpg'])
    print("After mpg conversion:", df.shape)
else:
    print("Column 'mpg' not found, skipping mpg conversion.")

# 7. Parse ENGINE column
# Extract engine size in liters (e.g. 1.5, 2.0, 3.5)
if 'engine' in df.columns:
    df['engine_size_l'] = df['engine'].str.extract(r'(\d\.\d)')
    df['engine_size_l'] = pd.to_numeric(df['engine_size_l'], errors='coerce')

    #Extract horsepower if present like "573HP"
    df['hp'] = df['engine'].str.extract(r'(\d+)\s*HP', flags=re.IGNORECASE)
    df['hp'] = pd.to_numeric(df['hp'], errors='coerce')

    df = df.drop(columns=['engine'])
    print("After engine parsing:", df.shape)
else:
    print("Column 'engine' not found, skipping engine parsing.")

# Remove records with 0 engine size
df = df[(df['engine_size_l'].notna()) & (df['engine_size_l'] != 0.0)]

# 8. Clean exterior_color textual noise
if 'exterior_color' in df.columns:
    df['exterior_color'] = (
        df['exterior_color']
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(r'[^a-z\s]', '', regex=True)  # remove digits, punctuation, codes
        .str.replace(r'\s+', ' ', regex=True)       # normalize spaces
    )
    print("After exterior_color cleaning:", df['exterior_color'].nunique())


# 8. Fill numeric columns with median
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

# 9. Save cleaned dataset
df.to_csv("cleaned_data.csv", index=False)
print("Cleaned dataset saved as 'cleaned_data.csv'")

Original shape: (762091, 20)
After percentile price filter: (731925, 20)
After percentile mileage filter: (702227, 20)
After year filter: (700791, 20)
After dropping useless/high-missing columns: (700791, 14)
Dropped columns: ['seller_name', 'seller_rating', 'price_drop', 'driver_rating', 'driver_reviews_num', 'interior_color']
After mpg conversion: (700791, 14)
After engine parsing: (700791, 15)
After exterior_color cleaning: 5515
Cleaned dataset saved as 'cleaned_data.csv'


In [13]:
df.head()

,manufacturer,model,year,mileage,transmission,drivetrain,fuel_type,exterior_color,accidents_or_damage,one_owner,personal_use_only,price,mpg_avg,engine_size_l,hp
0,acura,ILX Hybrid 1.5L,2013,92945.0,Automatic,Front-wheel Drive,Gasoline,black,0.0,0.0,0.0,13988.0,38.5,1.5,90.0
1,acura,ILX Hybrid 1.5L,2013,47645.0,Automatic CVT,Front-wheel Drive,Hybrid,gray,1.0,1.0,1.0,17995.0,38.5,1.5,172.0
2,acura,ILX Hybrid 1.5L,2013,53422.0,Automatic CVT,Front-wheel Drive,Hybrid,bellanova white pearl,0.0,1.0,1.0,17000.0,38.5,1.5,172.0
3,acura,ILX Hybrid 1.5L,2013,117598.0,Automatic CVT,Front-wheel Drive,Hybrid,polished metal metallic,0.0,1.0,1.0,14958.0,38.5,1.5,172.0
4,acura,ILX Hybrid 1.5L,2013,114865.0,Automatic CVT,Front-wheel Drive,Hybrid,nan,1.0,0.0,1.0,14498.0,38.5,1.5,172.0


## Data preprocessing

In [14]:
df.columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 647871 entries, 0 to 762090
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   manufacturer         647871 non-null  object 
 1   model                647871 non-null  object 
 2   year                 647871 non-null  int64  
 3   mileage              647871 non-null  float64
 4   transmission         645831 non-null  object 
 5   drivetrain           641550 non-null  object 
 6   fuel_type            638685 non-null  object 
 7   exterior_color       647871 non-null  object 
 8   accidents_or_damage  647871 non-null  float64
 9   one_owner            647871 non-null  float64
 10  personal_use_only    647871 non-null  float64
 11  price                647871 non-null  float64
 12  mpg_avg              647871 non-null  float64
 13  engine_size_l        647871 non-null  float64
 14  hp                   647871 non-null  float64
dtypes: float64(8), int64(1

### Mapping Fuel type, transmission, drive train

In [15]:
# 1. Mapping fuel values
fuel_mapping = {
    # Gasoline
    'Gas': 'gas',
    'Gasoline': 'gas',
    'Gasoline Fuel': 'gas',
    'Premium': 'gas',
    'Premium Unleaded': 'gas',
    'Premium (Required)': 'gas',
    'Regular Unleaded': 'gas',
    'G': 'gas',

    # Diesel
    'Diesel': 'diesel',
    'Diesel Fuel': 'diesel',
    'Bio Diesel': 'diesel',
    'Biodiesel': 'diesel',

    # Hybrid
    'Hybrid': 'hybrid',
    'Hybrid Fuel': 'hybrid',
    'Gas/Electric Hybrid': 'hybrid',
    'Gasoline/Mild Electric Hybrid': 'hybrid',
    'Plug-In Hybrid': 'hybrid',
    'Plug-In Electric/Gas': 'hybrid',
    'PHEV': 'hybrid',

    # Electric
    'Electric': 'electric',
    'Electric Fuel System': 'electric',

    # Flex fuel (E85)
    'E85 Fl': 'flex_fuel',
    'E85 Flex Fuel': 'flex_fuel',
    'Flex Fuel': 'flex_fuel',
    'Flex Fuel Capability': 'flex_fuel',
    'Flexible Fuel': 'flex_fuel',

    # Natural gas
    'Compressed Natural Gas': 'ng',
    'Natural Gas': 'ng',
    'Gaseous': 'ng',
    'Bi-Fuel': 'ng',

    # Other / unknown
    'Other': 'other',
    'Unspecified': 'other',
    'B': 'other',
}

df['fuel_type'] = df['fuel_type'].map(fuel_mapping)

print("After fuel_type normalization:", df['fuel_type'].value_counts())

# 2. Mapping transmission values
def clean_transmission(t):
    if pd.isna(t):
        return 'other'

    s = str(t).lower().strip()

    # manual
    if (
        'manual' in s or
        'm/t' in s or
        'tremec' in s or
        'stick' in s or
        'spd manual' in s
    ):
        return 'manual'

    # cvt / ivt / e-cvt / 
    if (
        'cvt' in s or
        'continuously variable' in s or
        'xtronic' in s or
        'lineartronic' in s or
        'ivt' in s or
        'ecvt' in s or
        'e-cvt' in s or
        'i-cvt' in s
    ):
        return 'cvt'

    # single-speed(mostly EVs)
    if (
        'single speed' in s or
        'single-speed' in s or
        'single speed reducer' in s or
        'single reduction' in s or
        'single-speed reduction' in s or
        'fixed gear' in s or
        'reduction gear' in s
    ):
        return 'single_speed'

    # everything else with "auto" etc. we treat as automatic
    if (
        'auto' in s or   
        'a/t' in s or
        'tiptronic' in s or
        'steptronic' in s or
        'speedshift' in s or
        'geartronic' in s or
        'dct' in s or
        'dual clutch' in s or
        'dual-clutch' in s or
        'pdk' in s or
        'torqshift' in s or
        'powershift' in s or
        'allison' in s or
        'skyactiv-drive' in s or
        'multispeed' in s or
        'multi-speed' in s
    ):
        return 'automatic'

    # fallback
    return 'other'


df['transmission'] = df['transmission'].apply(clean_transmission)

print("After transmission normalization:")
print(df['transmission'].value_counts())

# 3 Mapping drivetrain values
def clean_drivetrain(d):
    if pd.isna(d):
        return 'other'

    s = str(d).lower().strip()

    # Engine-text garbage that was scraped into this column by mistake
    if s.startswith('engine:'):
        return 'other'

    # 4WD / 4x4 / four-wheel drive
    if (
        '4wd' in s or
        '4x4' in s or
        'four wheel drive' in s or
        'four-wheel drive' in s or
        'four-wheel' in s
    ):
        return '4wd'

    # AWD / all-wheel drive
    if (
        'awd' in s or
        'all wheel drive' in s or
        'all-wheel drive' in s or
        'all-wheel' in s
    ):
        return 'awd'

    # FWD / front-wheel drive
    if (
        'fwd' in s or
        'front wheel drive' in s or
        'front-wheel drive' in s or
        'front-wheel' in s
    ):
        return 'fwd'

    # RWD / rear-wheel drive
    if (
        'rwd' in s or
        'rear wheel drive' in s or
        'rear-wheel drive' in s or
        'rear-wheel' in s
    ):
        return 'rwd'

    # 4x2 is ambiguous (2WD truck) – treat as RWD for now
    if '4x2' in s:
        return 'rwd'

    # Unknown / everything else
    if 'other' in s:
        return 'other'

    return 'other'


df['drivetrain'] = df['drivetrain'].apply(clean_drivetrain)

print("After drivetrain normalization:")
print(df['drivetrain'].value_counts())

After fuel_type normalization: fuel_type
gas          570701
hybrid        26325
diesel        23576
flex_fuel     16273
other          1278
electric        503
ng               28
Name: count, dtype: int64
After transmission normalization:
transmission
automatic       513121
cvt             109589
manual           18521
other             6639
single_speed         1
Name: count, dtype: int64
After drivetrain normalization:
drivetrain
fwd      222291
awd      201650
4wd      135711
rwd       81879
other      6340
Name: count, dtype: int64


### Map exterior color

In [16]:
BASE_COLORS = {
    "black", "white", "gray", "silver", "red", "blue", "green",
    "brown", "beige", "tan", "gold", "yellow", "orange", "purple"
}

PREMIUM_KEYWORDS = [
    "metal", "met", "metallic", "pearl", "pearlcoat", "mica",
    "coat", "tri", "tinted", "crystal", "diamond",
    "premium", "luxury", "magno", "matte", "satin"
]


def map_color(raw):
    if pd.isna(raw):
        return "other"

    s = str(raw).lower()

    # If a premium keyword exists anywhere → premium
    if any(pk in s for pk in PREMIUM_KEYWORDS):
        return "premium"

    # remove non-letters
    s_clean = re.sub(r'[^a-z ]', ' ', s)
    s_clean = re.sub(r'\s+', ' ', s_clean).strip()

    # find base colors
    for word in s_clean.split():
        for base in BASE_COLORS:
            if base in word:
                if base == "grey":
                    return "gray"
                if base == "tan":
                    return "beige"
                return base

    return "other"

df['exterior_color'] = df['exterior_color'].apply(map_color)
print(df['exterior_color'].value_counts())

exterior_color
premium    298907
white      102794
black       84296
gray        34215
silver      33497
other       32844
blue        25576
red         24056
green        2974
beige        2399
brown        1871
orange       1814
gold         1292
yellow       1079
purple        257
Name: count, dtype: int64


## Clean model column

In [17]:
print("\n Be patient! This might take long :/ ... ")

# Variants map
VARIANTS = [
    # Common trims
    "base", "s", "se", "sel", "ses",
    "sl", "slt", "sle",
    "sr", "sr5",
    "sx", "sxt",
    
    # Toyota
    "le", "xle", "xse",
    
    # Honda/Kia/Hyundai
    "lx", "ex", "lxs",
    
    # GM / Chevrolet trims
    "lt", "1lt", "2lt", "ltz", "ls",

    # Premium trims
    "premium",
    "limited",
    "platinum",
    "luxury",
    "touring",
    "reserve",
    "denali",
    "lariat",
    "titanium",

    # Performance / sport packages
    "sport",
    "gt",
    "turbo",
    "supercharged",
    "srt", "srt8",
    "scat",
    "gti",
    "raptor",
    "hellcat",
    
    # Jeep-related
    "latitude",
    "trailhawk",
    "sahara",
    "rubicon",
    "overland",
    "altitude",
    "laredo",

    # Ram-related
    "big horn",
    "tradesman",
    "longhorn",
    "rebel",
    "warlock",

    # Volvo
    "inscription",
    "momentum",
    
    # Misc valid trims
    "prestige",
    "classic",
    "preferred",
    "premier",
]


#  Extrat variants to model variant feature
def extract_variant(text):
    if pd.isna(text):
        return "other"

    s = str(text).lower()
    s = re.sub(r'[-/]', ' ', s)   # normalize separators, e.g. "XLE/SE"

    for v in VARIANTS:
        pattern = r"\b" + re.escape(v) + r"\b"
        if re.search(pattern, s):
            return v

    return "other"


# Clean model by removing drivetrain and engine size parameters
def clean_model(text):
    if pd.isna(text):
        return "other"

    s = str(text).lower()

    # normalize separators
    s = re.sub(r'[-/]', ' ', s)

    # remove drivetrain tokens
    s = re.sub(r"\b(awd|4wd|4x4|fwd|rwd)\b", " ", s)

    # remove pure liter engine sizes (2.0L.)
    s = re.sub(r"\b\d\.\d\s*l\b", " ", s)

    # remove turbo/engine codes (2.0t)
    s = re.sub(r"\b\d\.\d\s*t\b", " ", s)

    # remove numeric engine sizes (2.0)
    s = re.sub(r"\b\d\.\d\b", " ", s)

    # remove short “engine codes” like 2l, 3l
    s = re.sub(r"\b\d\s*l\b", " ", s)

    # remove extra spaces
    s = re.sub(r"\s+", " ", s).strip()

    return s if s else "other"


# 1 Extract variant from the model
df["model_variant"] = df["model"].apply(extract_variant)

# 2 Clean model 
df["model"] = df["model"].apply(clean_model)

# 3 Model clean
model_counts = df["model"].value_counts()
rare_models = model_counts[model_counts < 100].index
df["model"] = df["model"].replace(rare_models, "other")

# Check value counts
print("\n model_variant value_counts")
print(df["model_variant"].value_counts().head(30))

print("\n model value_counts (top 30) ===")
print(df["model"].value_counts().head(30))



 Be patient! This might take long :/ ... 

 model_variant value_counts
model_variant
other       178802
base         46657
se           34462
limited      33624
premium      31310
sport        31255
s            29272
touring      20028
ex           19030
lx           16949
sel          14609
lt           14369
le           11071
slt          10402
xle           9792
denali        8431
platinum      8271
sl            7570
gt            6967
sxt           6540
latitude      6482
luxury        6119
sle           5715
sr5           5440
1lt           5357
reserve       5244
lariat        4860
ls            4848
titanium      4316
big horn      4016
Name: count, dtype: int64

 model value_counts (top 30) ===
model
other                     88551
fusion se                  2997
corolla le                 2725
sportage lx                2705
sentra sv                  2543
optima lx                  2505
rogue sv                   2463
explorer xlt               2459
glc 300 base 4matic   

In [18]:
# Select only text/categorical columns
object_cols = df.select_dtypes(include=['object']).columns

unique_counts = {}

for col in object_cols:
    unique_counts[col] = df[col].nunique()

unique_counts = pd.Series(unique_counts).sort_values(ascending=False)

unique_counts

model             1281
model_variant       60
manufacturer        30
exterior_color      15
fuel_type            7
transmission         5
drivetrain           5
dtype: int64

### Feature engineering

In [19]:
df["car_age"] = 2023 - df["year"]

df["model_engine"] = df["model"] + "_" + df["engine_size_l"].astype(str)

df["model_drivetrain"] = df["model"] + "_" + df["drivetrain"].astype(str)

df["model_full"] = df["model"] +  "_" + df["engine_size_l"].astype(str) + "_" + df["drivetrain"].astype(str)

df.to_csv("preprocessed_data.csv", index=False)
print("Processed dataset saved as 'preprocessed_data.csv'")


Processed dataset saved as 'preprocessed_data.csv'


## Model training

In [20]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 1. Boolean types set as categories
bool_cols = ["accidents_or_damage", "one_owner", "personal_use_only"]

for col in bool_cols:
    if col in df.columns:
        df[col] = df[col].fillna(0).astype(int).astype("category")

# 2. Target
X = df.drop(columns=["price"])
y = df["price"]

# 3. Column types
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("Numeric:", numeric_cols)
print("Categorical:", categorical_cols)
print("\nData types:\n", df.dtypes)

# One-hot encoding
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

# 5. Time based train/test spit (90% / 10%) 
test_size = int(len(df) * 0.10)   # 10% test

X_train = X.iloc[:-test_size]
y_train = y.iloc[:-test_size]

X_test = X.iloc[-test_size:]
y_test = y.iloc[-test_size:]

print(f"\nTrain size: {len(X_train)}, Test size: {len(X_test)}")

# XGBOOST model Training and testing

xgb_best = XGBRegressor(
    objective="reg:squarederror",
    random_state=42,
    n_estimators=400,
    max_depth=8,
    learning_rate=0.03,
    subsample=1.0,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    n_jobs=-1,
)

xgb_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("xgb", xgb_best),
])

print("\n=== Training FINAL XGBoost with best params ===")
xgb_pipeline.fit(X_train, y_train)

# Cross vlaidation
cv = TimeSeriesSplit(n_splits=3)
xgb_cv_scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=cv, scoring="r2")
print("XGBoost CV R² scores:", xgb_cv_scores)
print("XGBoost CV R² mean:", xgb_cv_scores.mean())

# Test metrics
y_pred_xgb = xgb_pipeline.predict(X_test)

xgb_mae = mean_absolute_error(y_test, y_pred_xgb)
xgb_mse = mean_squared_error(y_test, y_pred_xgb)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r2 = r2_score(y_test, y_pred_xgb)

print("\n=== FINAL XGBOOST TEST METRICS ===")
print(f"MAE:  {xgb_mae:.2f}")
print(f"MSE:  {xgb_mse:.2f}")
print(f"RMSE: {xgb_rmse:.2f}")
print(f"R2:   {xgb_r2:.4f}")

# LIGHTGBM model training and testing 

lgbm_best = LGBMRegressor(
    objective="regression",
    random_state=42,
    n_estimators=200,      
    learning_rate=0.05,
    num_leaves=63,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    n_jobs=-1,
)

lgbm_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("lgbm", lgbm_best),
])

print("\n Training FINAL LightGBM with best params")
lgbm_pipeline.fit(X_train, y_train)

# Cross validation
lgbm_cv_scores = cross_val_score(lgbm_pipeline, X_train, y_train, cv=cv, scoring="r2")
print("LightGBM CV R² scores:", lgbm_cv_scores)
print("LightGBM CV R² mean:", lgbm_cv_scores.mean())

# Test metrics
y_pred_lgbm = lgbm_pipeline.predict(X_test)

lgbm_mae = mean_absolute_error(y_test, y_pred_lgbm)
lgbm_mse = mean_squared_error(y_test, y_pred_lgbm)
lgbm_rmse = np.sqrt(lgbm_mse)
lgbm_r2 = r2_score(y_test, y_pred_lgbm)

print("\n FINAL LIGHTGBM TEST METRICS")
print(f"MAE:  {lgbm_mae:.2f}")
print(f"MSE:  {lgbm_mse:.2f}")
print(f"RMSE: {lgbm_rmse:.2f}")
print(f"R2:   {lgbm_r2:.4f}")


# Baseline (Predicts the mean)
baseline_value = y_train.mean()
baseline_pred = np.full(shape=y_test.shape, fill_value=baseline_value)

# Metrics
baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_mse = mean_squared_error(y_test, baseline_pred)
baseline_rmse = np.sqrt(baseline_mse)
baseline_r2 = r2_score(y_test, baseline_pred)


# SIDE-BY-SIDE SUMMARY 

print("\n\n FINAL SIDE-BY-SIDE TEST COMPARISON")
print(f"Baseline -> MAE: {baseline_mae:.2f}, RMSE: {baseline_rmse:.2f}, R2: {baseline_r2:.4f}")
print(f"XGBoost  -> MAE: {xgb_mae:.2f}, RMSE: {xgb_rmse:.2f}, R2: {xgb_r2:.4f}")
print(f"LightGBM -> MAE: {lgbm_mae:.2f}, RMSE: {lgbm_rmse:.2f}, R2: {lgbm_r2:.4f}")


Numeric: ['year', 'mileage', 'mpg_avg', 'engine_size_l', 'hp', 'car_age']
Categorical: ['manufacturer', 'model', 'transmission', 'drivetrain', 'fuel_type', 'exterior_color', 'accidents_or_damage', 'one_owner', 'personal_use_only', 'model_variant', 'model_engine', 'model_drivetrain', 'model_full']

Data types:
 manufacturer             object
model                    object
year                      int64
mileage                 float64
transmission             object
drivetrain               object
fuel_type                object
exterior_color           object
accidents_or_damage    category
one_owner              category
personal_use_only      category
price                   float64
mpg_avg                 float64
engine_size_l           float64
hp                      float64
model_variant            object
car_age                   int64
model_engine             object
model_drivetrain         object
model_full               object
dtype: object

Train size: 583084, Test size: 64

In [24]:
def compute_grouped_importance(fi_df, categorical_cols, numeric_cols):
    rows = []

    # Numeric features: 1:1 mapping
    for col in numeric_cols:
        imp = fi_df.loc[fi_df["feature"] == col, "importance"].sum()
        rows.append({"feature": col, "importance": imp})

    # Categorical features: sum all OHE columns per original feature
    for col in categorical_cols:
        mask = fi_df["feature"].str.startswith(col + "_")
        imp = fi_df.loc[mask, "importance"].sum()
        rows.append({"feature": col, "importance": imp})

    grouped = pd.DataFrame(rows).sort_values(by="importance", ascending=False)
    return grouped

# Feature importance for XGBoost

print("\nXGBOOST feature importance")

# 1. Extract parts from the trained pipeline
pre = xgb_pipeline.named_steps["preprocess"]
xgb_model_only = xgb_pipeline.named_steps["xgb"]

# 2. Extract OHE-expanded categorical feature names
ohe = pre.named_transformers_["cat"]
ohe_feature_names = ohe.get_feature_names_out(categorical_cols)

# 3. Numeric features
numeric_feature_names = numeric_cols

# 4. Combine ALL feature names in the correct order
all_feature_names = list(ohe_feature_names) + list(numeric_feature_names)

# 5. Get feature importances
importance = xgb_model_only.feature_importances_

# 6. Build importance dataframe
fi_xgb = pd.DataFrame({
    "feature": all_feature_names,
    "importance": importance
}).sort_values(by="importance", ascending=False)

print("\nTop 50 Important Features (XGBoost) – raw OHE features:")
print(fi_xgb.head(50))

# Grouped importance (by original feature)
grouped_xgb = compute_grouped_importance(fi_xgb, categorical_cols, numeric_cols)

print("\nTop 20 Important Original Features (XGBoost):")
print(grouped_xgb.head(20))

print("\nLIGHTGBM Feature Importance")

pre_lgb = lgbm_pipeline.named_steps["preprocess"]
lgb_model_only = lgbm_pipeline.named_steps["lgbm"]

ohe_lgb = pre_lgb.named_transformers_["cat"]
ohe_feature_names_lgb = ohe_lgb.get_feature_names_out(categorical_cols)
numeric_feature_names_lgb = numeric_cols

all_features_lgb = list(ohe_feature_names_lgb) + list(numeric_feature_names_lgb)

importance_lgb = lgb_model_only.booster_.feature_importance(importance_type="gain")

importance_lgb = importance_lgb / importance_lgb.sum()

fi_lgb = pd.DataFrame({
    "feature": all_features_lgb,
    "importance": importance_lgb
}).sort_values(by="importance", ascending=False)

print("\nTop 50 Important Features (LightGBM) – raw OHE features:")
print(fi_lgb.head(50))

# Grouped importance (by original feature)
grouped_lgb = compute_grouped_importance(fi_lgb, categorical_cols, numeric_cols)

print("\nTop 20 Important Original Features (LightGBM):")
print(grouped_lgb.head(20))



XGBOOST feature importance

Top 50 Important Features (XGBoost) – raw OHE features:
                                         feature  importance
11068                              engine_size_l    0.026353
1186                              drivetrain_fwd    0.022783
1138                          model_x3 xdrive28i    0.021387
186                  model_bronco sport big bend    0.012262
1252                          model_variant_scat    0.012148
23                          manufacturer_porsche    0.010033
1189                            fuel_type_diesel    0.008449
1180                            transmission_cvt    0.008095
20                    manufacturer_mercedes-benz    0.007992
18                          manufacturer_lincoln    0.007707
2                               manufacturer_bmw    0.007289
8                              manufacturer_ford    0.007000
12                         manufacturer_infiniti    0.006893
1626            model_engine_amg gle 53 base_3.0    0.006817


In [ ]:
import joblib

# Save the LightGBM pipeline (with preprocessing inside)
joblib.dump(lgbm_pipeline, "used_car_lgbm_pipeline.pkl")
print("Saved model to used_car_lgbm_pipeline.pkl")


In [ ]:
models = sorted(df["mpg_avg"].dropna().unique().tolist())

for m in models:
    print(f'"{m}",')

In [ ]:
import json

# 1) Start from just the two columns and remove missing values
pairs = df[["manufacturer", "model"]].dropna()

# 2) Remove duplicates (same manufacturer/model combo)
pairs = pairs.drop_duplicates()

# 3) Build manufacturer -> [models] mapping
manufacturer_to_models = {}

for _, row in pairs.iterrows():
    brand = str(row["manufacturer"]).strip()
    model = str(row["model"]).strip()

    manufacturer_to_models.setdefault(brand, set()).add(model)

# Convert sets to sorted lists (JSON can’t handle sets)
manufacturer_to_models = {
    brand: sorted(list(models))
    for brand, models in manufacturer_to_models.items()
}

# 4) Build model -> manufacturer mapping (for autofill)
model_to_manufacturer = {}

for _, row in pairs.iterrows():
    brand = str(row["manufacturer"]).strip()
    model = str(row["model"]).strip()
    # If the same model appears under multiple brands, this keeps the last one
    model_to_manufacturer[model] = brand

# 5) Combine in one dict for export
mapping = {
    "manufacturerToModels": manufacturer_to_models,
    "modelToManufacturer": model_to_manufacturer,
}

# 6) Save to JSON file
with open("manufacturer_model_map.json", "w", encoding="utf-8") as f:
    json.dump(mapping, f, indent=2)